In [1]:
from spotify_functions import *
from get_album_counts import *

artist_name = "Beck"
artist_id = get_artist_id(artist_name, access_token)
if artist_id:
    album_count, total_tracks = get_album_counts(artist_id, access_token)
    print(f"Artist: {artist_name}")
    print(f"Number of Albums: {album_count}")
    print(f"Total Number of Tracks: {total_tracks}")
else:
    print(f"Artist {artist_name} not found.")

Artist: Beck
Number of Albums: 50
Total Number of Tracks: 312


In [6]:
from get_album_counts import *
from spotify_functions import *

def get_all_artists_album_count(csv_path, min_pause=0.5, max_pause=2.0, token_refresh_minutes=45):
    # Read the CSV file
    artists_df = pd.read_csv(csv_path)

    # Initialize list to store results
    album_data = []

    # Get initial access token and track time
    access_token = get_access_token()
    token_start_time = time.time()
    token_refresh_seconds = token_refresh_minutes * 60

    # Iterate through each artist with progress bar
    for idx, row in tqdm(artists_df.iterrows(), total=len(artists_df), desc="Fetching Album Counts"):
        # Check if we need to refresh the access token
        if time.time() - token_start_time > token_refresh_seconds:
            print(f"\nRefreshing access token after {token_refresh_minutes} minutes...")
            access_token = get_access_token()
            token_start_time = time.time()
            print("Access token refreshed successfully!")

        #artist_name = row['artist']
        artist_id = row['artist_id']

        # Get follower count
        try:
            albums = get_album_count(artist_id, access_token)
            album_data.append({
                #'artist': artist_name,
                'artist_id': artist_id,
                'album_count': album_count,
                'total_tracks': total_tracks
            })
        except Exception as e:
            print(f"Error fetching albums for {artist_id}: {e}")
            album_data.append({
                #'artist': artist_name,
                'artist_id': artist_id,
                'album_count': None,
                'total_tracks': None
            })

        # Random pause between requests (skip on last iteration)
        if idx < len(artists_df) - 1:
            pause_time = random.uniform(min_pause, max_pause)
            time.sleep(pause_time)

    # Create DataFrame from results
    followers_df = pd.DataFrame(album_data)

    return followers_df

In [7]:
#from spotify_functions import *
from get_album_counts import *

artist_id = get_all_artists_album_count("artist_followers.csv", access_token)

if artist_id:
    album_count, total_tracks = get_album_counts(artist_id, access_token)
    print(f"Artist: {artist_name}")
    print(f"Number of Albums: {album_count}")
    print(f"Total Number of Tracks: {total_tracks}")
else:
    print(f"Artist {artist_name} not found.")

Fetching Album Counts:   0%|          | 0/5121 [00:00<?, ?it/s]

Error fetching albums for 2rDxtYUzTAYJJE3Bl3Z5IN: get_album_count() takes 1 positional argument but 2 were given


TypeError: unsupported operand type(s) for -: 'float' and 'str'

### Get Album Counts for Artists in Batch (Final Code)

In [8]:
# python
import pandas as pd
import time
import random
from tqdm import tqdm

from spotify_functions import get_access_token, get_artist_id
from get_album_counts import get_album_counts

# Cell 1: single artist example
artist_name = "Beck"
access_token = get_access_token()
artist_id = get_artist_id(artist_name, access_token)
if artist_id:
    album_count, total_tracks = get_album_counts(artist_id, access_token)
    print(f"Artist: {artist_name}")
    print(f"Number of Albums: {album_count}")
    print(f"Total Number of Tracks: {total_tracks}")
else:
    print(f"Artist {artist_name} not found.")

# Cell 2: batch function
def get_all_artists_album_count(csv_path, min_pause=0.5, max_pause=2.0, token_refresh_minutes=45):
    artists_df = pd.read_csv(csv_path)
    album_data = []

    access_token = get_access_token()
    token_start_time = time.time()
    token_refresh_seconds = token_refresh_minutes * 60

    for idx, row in tqdm(artists_df.iterrows(), total=len(artists_df), desc="Fetching Album Counts"):
        # Refresh token if needed
        if time.time() - token_start_time > token_refresh_seconds:
            print(f"\nRefreshing access token after {token_refresh_minutes} minutes...")
            access_token = get_access_token()
            token_start_time = time.time()
            print("Access token refreshed successfully!")

        artist_id = row.get('artist_id') if 'artist_id' in row.index else None
        if not artist_id:
            # Skip or try to resolve from name
            artist_name = row.get('artist') if 'artist' in row.index else None
            if artist_name:
                artist_id = get_artist_id(artist_name, access_token)
        try:
            if artist_id:
                album_count, total_tracks = get_album_counts(artist_id, access_token)
            else:
                album_count, total_tracks = None, None

            album_data.append({
                'artist_id': artist_id,
                'album_count': album_count,
                'total_tracks': total_tracks
            })
        except Exception as e:
            print(f"Error fetching albums for {artist_id}: {e}")
            album_data.append({
                'artist_id': artist_id,
                'album_count': None,
                'total_tracks': None
            })

        # Pause between requests (skip on last iteration)
        if idx < len(artists_df) - 1:
            time.sleep(random.uniform(min_pause, max_pause))

    albums_df = pd.DataFrame(album_data)
    return albums_df

# Cell 3: run batch and inspect results
albums_df = get_all_artists_album_count("artist_followers.csv")
print(albums_df.head())

Artist: Beck
Number of Albums: 50
Total Number of Tracks: 312


Fetching Album Counts:   0%|          | 4/5121 [00:23<8:13:19,  5.78s/it]


KeyboardInterrupt: 